In [ ]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import tensorflow as tf
import os

def read_labeled_image_list(image_list_file):
    """Reads a .txt file containing pathes and labeles
    Args:
       image_list_file: a .txt file with one /path/to/image per line
       label: optionally, if set label will be pasted after each line
    Returns:
       List with all filenames in file image_list_file
    """
    f = open(image_list_file, 'r')
    filenames = []
    labels = []
    for line in f:
        filename, label = line[:-1].split(' ')
        filenames.append(filename)
        labels.append(int(label))
    return filenames, labels


X_Train = np.ones((1659,13,950,1)) # 166 * 9 165 * 1
Y_Train = np.zeros((1659,2))

X_Test = np.ones((166,13,950,1))
Y_Test = np.zeros((166,2))

file_list, label_list = read_labeled_image_list('trainlabel_mj.txt')


def getMFCC(filename):
    (rate,sig) = wav.read(filename)
    sig = sig[0:int(9.51*rate)]
    mfcc_feat = mfcc(sig,rate)
#    preprocess_mfcc = mfcc_feat-(np.mean(mfcc_feat, axis=0) + 1e-8)

#    fig, ax = plt.subplots()
    mfcc_data = np.swapaxes(mfcc_feat, 0, 1) #mfcc_feat

    return mfcc_data
#    print(mfcc_data)
#    cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower',aspect='auto')
#    ax.set_title('MFCC')


#    plt.show()

#    fig, ax = plt.subplots()
#    mfcc_data2 = np.swapaxes(preprocess_mfcc, 0, 1)
#    print(mfcc_data2)
#    cax = ax.imshow(mfcc_data2, interpolation='nearest', cmap=cm.coolwarm, origin='lower',aspect='auto')
#    ax.set_title('MFCC')

#    plt.show()
    
def get_train(image_list, label_list, idx):

    for i in range(64*(idx-1),64*idx) :
        tempary = getMFCC(image_list[i])
        temp_ = np.ones((13,950,1))
        for r in range(13):
            for c in range(950):
                temp_[r][c][0] = tempary[r][c]
        
        X_Train[i-(64*(idx-1))] = temp_
        Y_Train[i-(64*(idx-1))][label_list[i]] = 1
    return (X_Train, Y_Train)

def fill_train(image_list, label_list):
    for i in range(0, 1659):
        tempary = getMFCC(image_list[i])
        temp_ = np.ones((13,950,1))
        for r in range(13):
            for c in range(950):
                X_Train[i][r][c][0] = tempary[r][c]
        Y_Train[i][label_list[i]]=1
    
    return (X_Train, Y_Train)
        
        
def get_test(image_list, label_list, k):
    for i in range(166*k, 166*(k+1)):
        if(i==1659):
            continue

        tempary = getMFCC(image_list[i])
        temp_ = np.ones((13,950,1))
        for r in range(13):
            for c in range(950):
                temp_[r][c][0] = tempary[r][c]
        
        X_Test[i-(166*k)] = temp_
        Y_Test[i-(166*k)][label_list[i]] = 1
    return (X_Test, Y_Test)

# (X_Test, Y_Test) = get_test(file_list, label_list)
(X_Train, Y_Train) = fill_train(file_list, label_list)

batch_x = np.ones((64,13,950,1))
batch_y = np.zeros((64,2))

def get_batch(i, k):
    for idx in range(64*(i-1),64*i):
        
        if(idx>=166*k):
            real_idx = idx+166
        else:
            real_idx = idx
            
        batch_x[idx-(64*(i-1))] = X_Train[real_idx]
        batch_y[idx-(64*(i-1))] = Y_Train[real_idx]
    return (batch_x, batch_y)


In [ ]:
X = tf.placeholder('float', [None, 13, 950, 1])
Y = tf.placeholder('float', [None, 2])
keep_prob = tf.placeholder('float')

W1 = tf.get_variable("W11", shape=[2, 20, 1, 16],initializer=tf.contrib.layers.variance_scaling_initializer())
#W1 = tf.Variable(tf.random_normal([2, 20, 1, 64], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.maximum(0.2*L1, 0.01*L1)
L1 = tf.nn.max_pool(L1, ksize=[1,1,10,1], strides=[1,1,5,1], padding='SAME')  #13*190*16

print("L1.shape : ",L1.shape)


W2 = tf.get_variable("W21", shape=[2, 10, 16, 32],initializer=tf.contrib.layers.variance_scaling_initializer())
#W2 = tf.Variable(tf.random_normal([2,10,64,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.maximum(0.2*L2, 0.01*L2)
L2 = tf.nn.max_pool(L2, ksize=[1,1,7,1], strides=[1,1,5,1], padding='SAME')   #13*38*32

print("L2.shape : ",L2.shape)

#W3 = tf.get_variable("W3", shape=[2, 5, 64, 64],initializer=tf.contrib.layers.variance_scaling_initializer())
##W3 = tf.Variable(tf.random_normal([2,5,64,64], stddev = 0.01))
#L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
#L3 = tf.maximum(0.3*L3, 0.05*L3)
#L3 = tf.nn.max_pool(L3, ksize=[1,1,4,1], strides=[1,1,2,1], padding='SAME')   #13*10*64

#print("L3.shape : ",L3.shape)
11
W4 = tf.get_variable("W41", shape=[13*38*32, 1024],initializer=tf.contrib.layers.variance_scaling_initializer())
#W4 = tf.Variable(tf.random_normal([7*10*64, 1024], stddev=0.01))
L4 = tf.reshape(L2, [-1, 13*38*32])
L4 = tf.matmul(L4, W4)
L4 = tf.maximum(0.2*L4, 0.01*L4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.get_variable("W51", shape=[1024, 256],initializer=tf.contrib.layers.variance_scaling_initializer())
#W5 = tf.Variable(tf.random_normal([1024, 256], stddev=0.01))
L5 = tf.matmul(L4, W5)
L5 = tf.maximum(0.2*L5, 0.01*L5)
L5 = tf.nn.dropout(L5, keep_prob)

W6 = tf.get_variable("W61", shape=[256, 2],initializer=tf.contrib.layers.variance_scaling_initializer())
#W6 = tf.Variable(tf.random_normal([256, 2], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L5,W6))#tf.nn.softmax(tf.matmul(L5, W6))

#W3 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
#L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
#L3 = tf.nn.relu(L3)
#L3 = tf.nn.max_pool(L3, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')    #43*28*64

#W4 = tf.Variable(tf.random_normal([3,3,64,64], stddev = 0.01))
#L4 = tf.nn.conv2d(L3, W4, strides=[1,1,1,1], padding='SAME')
#L4 = tf.nn.relu(L4)
#L4 = tf.nn.max_pool(L4, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')     #22*14*64

#W5 = tf.Variable(tf.random_normal([3,3,64,64], stddev = 0.01))
#L5 = tf.nn.conv2d(L4, W5, strides=[1,1,1,1], padding='SAME')
#L5 = tf.nn.relu(L5)
#L5 = tf.nn.max_pool(L5, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')     #11*7*64


#W6 = tf.Variable(tf.random_normal([7*11*64, 256], stddev=0.01))
#L6 = tf.reshape(L5, [-1,7*11*64])
#L6 = tf.matmul(L6, W6)
#L6 = tf.nn.relu(L6)
#L6 = tf.nn.dropout(L6, keep_prob)

#W7 = tf.Variable(tf.random_normal([256,64], stddev=0.01))
#L7 = tf.matmul(L6, W7)
#L7 = tf.nn.relu(L7)


#W8 = tf.Variable(tf.random_normal([64, 2], stddev=0.01))
#model = tf.matmul(L7, W8)

model_clipped = tf.clip_by_value(model, 1e-10, 0.9999999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(Y*tf.log(model_clipped)+(1-Y)*tf.log(1-model_clipped),axis=1))


optimizer = tf.train.AdamOptimizer(learning_rate=0.000000037).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 5
total_batch = int(1494/64)
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(model,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, 'float'))

TP = tf.count_nonzero(tf.argmax(model,1) * tf.argmax(Y,1))
TN = tf.count_nonzero((tf.argmax(model,1)-1) * (tf.argmax(Y,1)-1))
FP = tf.count_nonzero(tf.argmax(model,1) * (tf.argmax(Y ,1)-1))
FN = tf.count_nonzero((tf.argmax(model,1)-1) * tf.argmax(Y,1))

for k in range(0, 10):
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(init)
        for epoch in range(2000):
            total_cost = 0
            for i in range(total_batch):
                #(x_data, y_data) = get_train(file_list, label_list, i+1)
                (x_data, y_data) = get_batch(i+1, k)
                _, cost_val = sess.run([optimizer, cross_entropy], feed_dict={X:x_data, Y:y_data, keep_prob:0.7})
                total_cost += cost_val / total_batch
            if(epoch%100 == 0 or epoch%100 == 1):
                (X_Test, Y_Test) = get_test(file_list, label_list, k)
                print('Epoch: %04d' % (epoch+1),
                     'Avg. cost = %.3f' % (total_cost))
                print('accuracy: ',sess.run([accuracy,model,Y], feed_dict={X:X_Test, Y:Y_Test, keep_prob:1}))
                print('TP: ',sess.run(TP,feed_dict={X:X_Test, Y:Y_Test, keep_prob:1}))
                print('TN: ',sess.run(TN,feed_dict={X:X_Test, Y:Y_Test, keep_prob:1}))
                print('FP: ',sess.run(FP,feed_dict={X:X_Test, Y:Y_Test, keep_prob:1}))
                print('FN: ',sess.run(FN,feed_dict={X:X_Test, Y:Y_Test, keep_prob:1}))
    
        print('complete optimization')




        print('accuracy: ',sess.run([accuracy,model,Y], feed_dict={X:X_Test, Y:Y_Test, keep_prob:1}))